In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    sys.path.insert(0, '../../')
sys.path.insert(0, '../../')
from src.data.preprocessing import *
from src.data.load_data_yoochoose import *
from src.models.model import LSTMRec
from src.models import train
from keras.optimizers import Adam
from keras.metrics import Precision, Recall
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

Using TensorFlow backend.


In [11]:
def train_and_validate(train_x, train_y, test_x, test_y, hparams):
    unique_items = len(train_y[0])
    model = LSTMRec(
        vocabulary_size=unique_items,
        emb_output_dim=hparams['emb_dim'],
        lstm_units=hparams['lstm_units'],
        lstm_activation=hparams['lstm_activation'],
        lstm_recurrent_activation=hparams['lstm_recurrent_activation'],
        lstm_dropout=hparams['lstm_dropout'],
        lstm_recurrent_dropout=hparams['lstm_recurrent_dropout'],
        dense_activation=hparams['dense_activation']
    )
    model.compile(
        optimizer=Adam(
            learning_rate=hparams['learning_rate'],
            beta_1=hparams['adam_beta_1'],
            beta_2=hparams['adam_beta_2'],
            epsilon=hparams['adam_epsilon']
        ),
        loss='binary_crossentropy',
        metrics=[
            Precision(top_k=1, name='P_at_1'),
            Precision(top_k=3, name='P_at_3'),
            Precision(top_k=5, name='P_at_5'),
            Precision(top_k=10, name='P_at_10'),
            Recall(top_k=10, name='R_at_10'),
            Recall(top_k=50, name='R_at_50'),
            Recall(top_k=100, name='R_at_100')
        ]
    )
    hst = model.fit(
        x=train_x,
        y=train_y,
        batch_size=hparams['batch_size'],
        epochs=3,
        callbacks=[
            EarlyStopping(
                monitor='val_R_at_10',
                patience=10,
                mode='max',
                restore_best_weights=True
            ),
            ModelCheckpoint(
                filepath='../../models/' + hparams['run_id'] + '.ckpt',
                monitor='val_R_at_10',
                mode='max',
                save_best_only=True,
                save_weights_only=True
            ),
            TensorBoard(
                log_dir='../../logs/' + hparams['run_id'],
                histogram_freq=1
            )
        ],
        validation_split=0.2
    )

    val_best_epoch = np.argmax(hst.history['val_R_at_10'])
    test_results = model.evaluate(test_x, test_y)
    with tf.summary.create_file_writer('../../logs/' + hparams['run_id']).as_default():
        hp.hparams(hparams)
        tf.summary.scalar('train.final_loss', hst.history["val_loss"][val_best_epoch], step=0)
        tf.summary.scalar('train.final_P_at_1', hst.history["val_P_at_1"][val_best_epoch], step=0)
        tf.summary.scalar('train.final_P_at_3', hst.history["val_P_at_3"][val_best_epoch], step=0)
        tf.summary.scalar('train.final_P_at_5', hst.history["val_P_at_5"][val_best_epoch], step=0)
        tf.summary.scalar('train.final_P_at_10', hst.history["val_P_at_10"][val_best_epoch], step=0)
        tf.summary.scalar('train.final_R_at_10', hst.history["val_R_at_10"][val_best_epoch], step=0)
        tf.summary.scalar('train.final_R_at_50', hst.history["val_R_at_50"][val_best_epoch], step=0)
        tf.summary.scalar('train.final_R_at_100', hst.history["val_R_at_100"][val_best_epoch], step=0)

        tf.summary.scalar('train.final_loss', test_results[0], step=0)
        tf.summary.scalar('train.final_P_at_1', test_results[1], step=0)
        tf.summary.scalar('train.final_P_at_3', test_results[2], step=0)
        tf.summary.scalar('train.final_P_at_5', test_results[3], step=0)
        tf.summary.scalar('train.final_P_at_10', test_results[4], step=0)
        tf.summary.scalar('train.final_R_at_10', test_results[5], step=0)
        tf.summary.scalar('train.final_R_at_50', test_results[6], step=0)
        tf.summary.scalar('train.final_R_at_100', test_results[7], step=0)

In [3]:
hparams = {}
hparams['emb_dim'] = 50 * np.random.randint(1, 11)
hparams['lstm_units'] = 25 * np.random.randint(1, 13)
hparams['lstm_activation'] = ['relu', 'sigmoid', 'tanh', 'linear', 'softmax'][np.random.randint(0, 5)]
hparams['lstm_recurrent_activation'] = ['relu', 'sigmoid', 'tanh', 'linear', 'softmax'][np.random.randint(0, 5)]
hparams['lstm_dropout'] = 0.05 * np.random.randint(1, 11)
hparams['lstm_recurrent_dropout'] = 0.05 * np.random.randint(1, 11)
hparams['dense_activation'] = ['relu', 'sigmoid', 'tanh', 'linear', 'softmax'][np.random.randint(0, 5)]
hparams['batch_size'] = 2 ** (np.random.randint(3, 11))
hparams['learning_rate'] = 10 ** (-np.random.randint(2, 5))
hparams['adam_beta_1'] = 0.05 * (np.random.randint(14, 25))
hparams['adam_beta_2'] = 0.05 * (np.random.randint(14, 25)) - 0.001
hparams['adam_epsilon'] = 10 ** (-np.random.randint(6, 11))
hparams['run_id'] = 'test'

In [4]:
data_x, data_y = load_processed_sparse()
data_y = data_y.toarray()
data_x = data_x[:100]
data_y = data_y[:100]
train_x, test_x, train_y, test_y = data_split(data_x, data_y)
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(80, 40)
(80, 23091)
(20, 40)
(20, 23091)


In [12]:
train_and_validate(train_x, train_y, test_x, test_y, hparams)

Train on 64 samples, validate on 16 samples
Epoch 1/3
64/64 [==============================] - 1s 16ms/step - loss: 0.0039 - P_at_1: 0.0000e+00 - P_at_3: 0.0000e+00 - P_at_5: 0.0000e+00 - P_at_10: 0.0000e+00 - R_at_10: 0.0000e+00 - R_at_50: 0.0032 - R_at_100: 0.0032 - val_loss: 0.0040 - val_P_at_1: 0.0000e+00 - val_P_at_3: 0.0000e+00 - val_P_at_5: 0.0000e+00 - val_P_at_10: 0.0000e+00 - val_R_at_10: 0.0000e+00 - val_R_at_50: 0.0000e+00 - val_R_at_100: 0.0000e+00
Epoch 2/3
64/64 [==============================] - 1s 9ms/step - loss: 0.0039 - P_at_1: 0.0000e+00 - P_at_3: 0.0000e+00 - P_at_5: 0.0000e+00 - P_at_10: 0.0000e+00 - R_at_10: 0.0000e+00 - R_at_50: 0.0032 - R_at_100: 0.0032 - val_loss: 0.0040 - val_P_at_1: 0.0000e+00 - val_P_at_3: 0.0000e+00 - val_P_at_5: 0.0000e+00 - val_P_at_10: 0.0000e+00 - val_R_at_10: 0.0000e+00 - val_R_at_50: 0.0000e+00 - val_R_at_100: 0.0000e+00
Epoch 3/3
20/20 [==============================] - 0s 5ms/step


In [10]:
# Load the TensorBoard notebook extension
log_dir = '..\\..\\logs'
%load_ext tensorboard
%tensorboard --logdir {log_dir}

Reusing TensorBoard on port 6006 (pid 2864), started 0:51:57 ago. (Use '!kill 2864' to kill it.)